In [1]:
import os
import pandas as pd
import json
import janitor

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

from utilities import read_jsons

In [2]:
# Read in 100 random sample 
df_random_sample = (
    pd.read_csv("../input/random_sample.csv")
    .assign(slug=lambda df: df["github"].str.replace("https://github.com/", ""))
    .assign(slug=lambda df: df["slug"].apply(lambda x: x[:-1] if x.endswith("/") else x))
    .assign(slug=lambda df: df["slug"].str.lower().str.strip())
    .assign(treated=1)
)
df_random_sample.head(3)

,pkg,return_code,github_url,homepage,earliest_release,gh_url_check,github,pypi,label,slug,treated
0,bird-ospf-link-db-parser,200.0,https://github.com/Andrew-Dickinson/bird-ospf-...,NaN,2023-04-29T07:23:44,1.0,https://github.com/Andrew-Dickinson/bird-ospf-...,https://pypi.org/project/bird-ospf-link-db-par...,bird-ospf-link-db-parser\nhttps://github.com/A...,andrew-dickinson/bird-ospf-link-db-parser,1
1,asciicli,200.0,https://github.com/mrq-andras/asciicli,https://github.com/mrq-andras/asciicli,2023-04-28T07:22:55,1.0,https://github.com/mrq-andras/asciicli,https://pypi.org/project/asciicli/#history,asciicli\nhttps://github.com/mrq-andras/asciic...,mrq-andras/asciicli,1
2,bdpotentiometer,200.0,https://github.com/bond-anton/BDPotentiometer,https://github.com/bond-anton/BDPotentiometer,2023-04-27T06:35:18,1.0,https://github.com/bond-anton/BDPotentiometer,https://pypi.org/project/bdpotentiometer/#history,bdpotentiometer\nhttps://github.com/bond-anton...,bond-anton/bdpotentiometer,1


In [3]:
# Repos characteristics
payloads = read_jsons("../output/repo_profile_payload/")

df_repos = (
    pd.DataFrame(payloads)
    .dropna(subset=["id"], ignore_index=True)
    .assign(slug=lambda df: df["full_name"].str.lower().str.strip())
    .assign(user=lambda df: [slug.split("/")[0] for slug in df["slug"]])
    .assign(license_str=lambda df: [license["spdx_id"] if license else None for license in df["license"]])
    .assign(owner_str=lambda df: [owner["login"].lower().strip() if owner else None for owner in df["owner"]])
    .merge(df_random_sample, how="left", on="slug", validate="1:1", indicator=True)
    .assign(treated=lambda df: df["treated"].fillna(0).apply(int))
    .assign(created_at=lambda df: pd.to_datetime(df['created_at']))
    .assign(year_created=lambda df: [dt.year for dt in df["created_at"]])
)
assert (df_repos["user"] == df_repos["owner_str"]).all()
df_repos.head()

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,forks_url,keys_url,collaborators_url,teams_url,hooks_url,issue_events_url,events_url,assignees_url,branches_url,tags_url,blobs_url,git_tags_url,git_refs_url,trees_url,statuses_url,languages_url,stargazers_url,contributors_url,subscribers_url,subscription_url,commits_url,git_commits_url,comments_url,issue_comment_url,contents_url,compare_url,merges_url,archive_url,downloads_url,issues_url,pulls_url,milestones_url,notifications_url,labels_url,releases_url,deployments_url,created_at,updated_at,pushed_at,git_url,...,stargazers_count,watchers_count,language,has_issues,has_projects,has_downloads,has_wiki,has_pages,has_discussions,forks_count,mirror_url,archived,disabled,open_issues_count,license,allow_forking,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,permissions,temp_clone_token,network_count,subscribers_count,organization,parent,source,message,documentation_url,template_repository,slug,user,license_str,owner_str,pkg,return_code,github_url,homepage_y,earliest_release,gh_url_check,github,pypi,label,treated,_merge,year_created
0,629516002.0,R_kgDOJYWm4g,dustpylib,stammler/dustpylib,False,"{'login': 'stammler', 'id': 21321343, 'node_id...",https://github.com/stammler/dustpylib,Library with auxiliary modules for DustPy,False,https://api.github.com/repos/stammler/dustpylib,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,https://api.github.com/repos/stammler/dustpyli...,2023-04-18 13:22:25+00:00,2023-04-27T17:59:43Z,2023-05-19T21:00:15Z,git://github.com/stammler/dustpylib.git,...,0.0,0.0,Python,True,True,True,True,False,False,0.0,NaN,False,False,0.0,"{'key': 'bsd-3-clause', 'name': 'BSD 3-Clause ...",True,False,False,"[astrophysics, planetary-science, python]",public,0.0,0.0,0.0,main,"{'admin': False, 'maintain': False, 'push': Fa...",,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,stammler/dustpylib,stammler,BSD-3-Clause,stammler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,left_only,2023
1,633593525.0,R_kgDOJcPetQ,enerbitdso,enerBit/enerbitdso,False,"{'login': 'enerBit', 'id': 97300957, 'node_id'...",https://github.com/enerBit/enerbitdso,None,False,https://api.github.com/repos/enerBit/enerbitdso,https://api.github.com/repos/enerBit/enerbitds...,https://api.github

In [4]:
# Users characteristics
df_users = (
    pd.read_csv("../output/users_profile.csv")
    .assign(user=lambda df: df["login"].str.lower().str.strip())
    .drop_duplicates("user")
    .merge((
        df_repos
        .select_columns(["user", "treated"])
        .drop_duplicates(["user", "treated"])
    ), how="left", on="user", validate="1:m"
    )
    .assign(treated=lambda df: df["treated"].fillna(0).apply(int))    
    .assign(created_at=lambda df: pd.to_datetime(df['created_at']))
    .assign(year_created=lambda df: [dt.year for dt in df["created_at"]])    
    .assign(updated_at=lambda df: pd.to_datetime(df['created_at']))
    .assign(year_updated=lambda df: [dt.year for dt in df["updated_at"]])
)
df_users.head()

,login,id,node_id,avatar_url,gravatar_id,url,html_url,followers_url,following_url,gists_url,starred_url,subscriptions_url,organizations_url,repos_url,events_url,received_events_url,type,site_admin,name,company,blog,location,email,hireable,bio,twitter_username,public_repos,public_gists,followers,following,created_at,updated_at,retrieval_date,user,treated,year_created,year_updated
0,Arsybai,33319709,MDQ6VXNlcjMzMzE5NzA5,https://avatars.githubusercontent.com/u/333197...,NaN,https://api.github.com/users/Arsybai,https://github.com/Arsybai,https://api.github.com/users/Arsybai/followers,https://api.github.com/users/Arsybai/following...,https://api.github.com/users/Arsybai/gists{/gi...,https://api.github.com/users/Arsybai/starred{/...,https://api.github.com/users/Arsybai/subscript...,https://api.github.com/users/Arsybai/orgs,https://api.github.com/users/Arsybai/repos,https://api.github.com/users/Arsybai/events{/p...,https://api.github.com/users/Arsybai/received_...,User,False,Arsybai,Clee Ltd,http://arsybai.com,indonesia,me@arsybai.com,NaN,Just want to death by cuteness\r\n,NaN,36,0,64,7,2017-11-02 12:38:27+00:00,2017-11-02 12:38:27+00:00,2023-05-20 21:41:26.180953,arsybai,0,2017,2017
1,GaNiziolek,54728889,MDQ6VXNlcjU0NzI4ODg5,https://avatars.githubusercontent.com/u/547288...,NaN,https://api.github.com/users/GaNiziolek,https://github.com/GaNiziolek,https://api.github.com/users/GaNiziolek/followers,https://api.github.com/users/GaNiziolek/follow...,https://api.github.com/users/GaNiziolek/gists{...,https://api.github.com/users/GaNiziolek/starre...,https://api.github.com/users/GaNiziolek/subscr...,https://api.github.com/users/GaNiziolek/orgs,https://api.github.com/users/GaNiziolek/repos,https://api.github.com/users/GaNiziolek/events...,https://api.github.com/users/GaNiziolek/receiv...,User,False,Gabriel Niziolek,@TempoX-Ltda,NaN,Brazil,NaN,NaN,Software Developer on TempoX | Django | Pyrami...,gniziolek,25,3,6,15,2019-08-31 00:19:09+00:00,2019-08-31 00:19:09+00:00,2023-05-20 21:41:26.180953,ganiziolek,1,2019,2019
2,MihailSalnikov,2613180,MDQ6VXNlcjI2MTMxODA=,https://avatars.githubusercontent.com/u/261318...,NaN,https://api.github.com/users/MihailSalnikov,https://github.com/MihailSalnikov,https://api.github.com/users/MihailSalnikov/fo...,https://api.github.com/users/MihailSalnikov/fo...,https://api.github.com/users/MihailSalnikov/gi...,https://api.github.com/users/MihailSalnikov/st...,https://api.github.com/users/MihailSalnikov/su...,https://api.github.com/users/MihailSalnikov/orgs,https://api.github.com/users/MihailSalnikov/repos,https://api.github.com/users/MihailSalnikov/ev...,https://api.github.com/users/MihailSalnikov/re...,User,False,Mikhail Salnikov,NLP Group,https://medium.com/@MSalnikov,NaN,NaN,True,Researcher in DL and NLP,NaN,27,24,7,2,2012-10-21 15:03:02+00:00,2012-10-21 15:03:02+00:00,2023-05-20 21:41:26.180953,mihailsalnikov,0,2012,2012
3,Simply-Artificial,130693929,O_kgDOB8o7KQ,https://avatars.githubusercontent.com/u/130693...,NaN,https://api.github.com/users/Simply-Artificial,https://github.com/Simply-Artificial,https://api.github.com/users/Simply-Artificial...,https://api.github.com/users/Simply-Artificial...,https://api.github.com/users/Simply-Artificial...,https://api.github.com/users/Simply-Artificial...,https://api.github.com/users/Simply-Artificial...,https://api.github.com/users/Simply-Artificial...,https://api.github.com/users/Simply-Artificial...,https://api.github.com/users/Simply-Artificial...,https://api.github.com/users/Simply-Artificial...,Organization,False,Simply Artificial,NaN,NaN,NaN,simply-artificial@itsmealfie0.com,NaN,NaN,NaN,2,0,1,0,2023-04-13 17:37:21+00:00,2023-04-13 17:37:21+00:00,2023-05-20 21:41:26.180953,simply-artificial,0,2023,2023
4,jposada202020,34255413,MDQ6VXNlcjM0MjU1NDEz,https://avatars.githubusercontent.com/u/342554...,NaN,https://api.github.com/users/jposada202020,https://github.com/jposada202020,https://api.github.com/users/jposada202020/fol...,https://api.github.com/users/jposada202020

In [5]:
df_repos.to_csv("../output/repo_baselines.csv", index=False)
df_users.to_csv("../output/user_baselines.csv", index=False)